# Mini-projeto 07 — EDA avançado

Qualidade + segmentação + narrativa (insights exportáveis).

> Entregável de portfólio — gerado em 2026-01-30.


## Objetivo

        Aplicar checks de qualidade, detectar outliers e criar uma segmentação simples (RFM) com narrativa em Markdown.

        ## Entregáveis (para portfólio)

        - [ ] `reports/eda_avancado_story.md`
- [ ] `assets/pareto_produtos.png`

        ## Como usar

        1- Rode este notebook (kernel com o `.venv` do repo)  
        2- Gere **assets** (imagens/HTML) e **reports** (markdown/json) dentro desta pasta  
        3- Faça commit dos arquivos gerados para evidenciar o resultado no GitHub

In [ ]:
# Setup: detectar raiz do repositório + paths padrão
from pathlib import Path
import pandas as pd
import numpy as np

def find_repo_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for _ in range(12):
        if (p / "requirements.txt").exists() and (p / "README.md").exists():
            return p
        p = p.parent
    return (start or Path.cwd()).resolve()

ROOT = find_repo_root()
DATA_SAMPLE = ROOT / "data" / "sample"
DATA_SOURCE = ROOT / "data" / "source" / "bases-dados-analytics-powerbi-ml"

# Pasta do projeto (onde salvar assets/outputs/reports)
PROJ = ROOT / "projects" / "07_eda_avancado"
ASSETS = PROJ / "assets"
OUTPUTS = PROJ / "outputs"
REPORTS = PROJ / "reports"
for d in (ASSETS, OUTPUTS, REPORTS):
    d.mkdir(parents=True, exist_ok=True)

# Dataset padrão (sempre disponível)
sales_path = DATA_SAMPLE / "sales.csv"
customers_path = DATA_SAMPLE / "customers.csv"
sales = pd.read_csv(sales_path, parse_dates=["date"])
customers = pd.read_csv(customers_path, parse_dates=["signup_date"])

# Dataset real (opcional): se você adicionou o submodule/clone em dados/source/
has_real = DATA_SOURCE.exists()

print("ROOT:", ROOT)
print("Dataset sample:", sales.shape, customers.shape)
print("Dataset real disponível?", has_real)

In [ ]:
import matplotlib.pyplot as plt

df = sales.copy()
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# 1) Qualidade: valores negativos, nulos, tipos
checks = {
    "date_nulls": int(df["date"].isna().sum()),
    "revenue_negative": int((df["revenue"] < 0).sum()),
    "qty_nonpositive": int((df["qty"] <= 0).sum()),
}
print("Checks:", checks)

# 2) Outliers simples via IQR
q1, q3 = np.percentile(df["revenue"], [25, 75])
iqr = q3 - q1
upper = q3 + 1.5 * iqr
outliers = df[df["revenue"] > upper]

# 3) Pareto de produtos (top N por receita)
by_prod = df.groupby("product", as_index=False)["revenue"].sum().sort_values("revenue", ascending=False)
top = by_prod.head(10)
plt.figure()
plt.bar(top["product"], top["revenue"])
plt.title("Top 10 produtos por receita")
plt.xticks(rotation=30, ha="right")
out_img = ASSETS / "pareto_produtos.png"
plt.tight_layout()
plt.savefig(out_img, dpi=160)
plt.close()
print("Salvo:", out_img)

# 4) RFM simples (por customer_id)
now = df["date"].max()
rfm = (
    df.groupby("customer_id")
    .agg(
        recency=("date", lambda s: (now - s.max()).days),
        frequency=("order_id", "nunique"),
        monetary=("revenue", "sum"),
    )
    .reset_index()
)

# Score simples por quartis
rfm["r_score"] = pd.qcut(rfm["recency"], 4, labels=[4,3,2,1]).astype(int)
rfm["f_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 4, labels=[1,2,3,4]).astype(int)
rfm["m_score"] = pd.qcut(rfm["monetary"].rank(method="first"), 4, labels=[1,2,3,4]).astype(int)
rfm["rfm_score"] = rfm["r_score"] + rfm["f_score"] + rfm["m_score"]

vip = rfm.sort_values("rfm_score", ascending=False).head(10)

md = []
md.append("# EDA avançado — Story\n")
md.append("## Checks de qualidade\n")
md.append("```json\n" + json.dumps(checks, indent=2, ensure_ascii=False) + "\n```\n")
md.append(f"## Outliers (revenue > {upper:,.2f})\n- Linhas: {len(outliers)}\n")
md.append("## Top 10 clientes (RFM score)\n")
md.append(vip.to_markdown(index=False))
out = REPORTS / "eda_avancado_story.md"
out.write_text("\n".join(md), encoding="utf-8")
print("Salvo:", out)

## Evidências

![](./assets/pareto_produtos.png)
